# EMit - Emotions in Italian
## Emotion and Target detection
### Emanuele Muzio - 0766230
Il presente notebook rappresenta un'illustrazione dello svolgimento dei due task del contest EMit (Emotions in Italian):
- il primo task, o Task A, è mirato alla classificazione dei sentimenti espressi dai messaggi sui social media riguardanti i prodotti RAI
- il secondo task, o Task B, è mirato alla classificazione del target del messaggio, se sul topic o sulla direzione
Entrambi i task sono classificazione multiclasse e multilabel (nel Task A abbiamo 8 emozioni di base, più _Love_, nel Task B il target può essere uno dei due, entrambi o nessuno).

Ai fini della valutazione dei due task, sono stati forniti dei dataset da utilizzare sia per l'addestramento che per i test, comprensivi di annotazioni.

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import copy, os
import numpy as np
from sklearn.metrics import classification_report as cr, f1_score as f1
import seaborn as sns
import random as rand
import pickle 
import matplotlib.image as mpimg

#Import torch
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from torchmetrics.classification import MultilabelAccuracy
from transformers import AutoTokenizer, BertModel

# Gestione delle emoji
import emot as emot_imp
emot = emot_imp.core.emot()

## Task A: Emotion detection

Per andare a ricercare la massima precisione (sia per quanto riguarda i dati che il modello), verranno effettuate diverse prove:
- dati: verranno effettuati dei test usando sia dei dati preprocessati che i dati originali (dal momento che il blocco BERT-like del nostro modello trae informazioni anche da punteggiatura, stopwords ecc...)
- modello: verranno testati diverse combinazioni di iperparametri usando un approccio grid-like per trovare il modello migliore

Per il primo task, procediamo in due step, partendo dal preprocessing del testo (rimozione stopwords, hashtag, menzioni, keyword RT, collegamenti ipertestuali) per poi andare alla classificazione.

Per la tokenizzazione del testo è stato scelto un modello multilingua (dal momento che i messaggi riguardano un'emittente italiana) case-sensitive. 

Il supporto alle emoji/emoticons è stato aggiunto rilevando quelle presenti nel testo e facendo un resize del modello per poterle gestire.

Il modello BERT-like da HuggingFace utilizzato è il seguente: _bert-base-multilingual-cased_.

In [ ]:
data = pd.read_csv("release\emit_2023-v1\emit_train_A.csv")
text_prep = data["text"]
text = data["text"]
emotions = data.drop(["id","text"], axis=1)
emotions_prep = data.drop(["id","text"], axis=1)
model_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
nltk.download('stopwords')
stopwords_it = stopwords.words('italian')
X_keywords = ['RT', 'FAV']

In [ ]:
# Funzione che raccoglie i principali 4 controlli che vengono effettuati sul testo, ovvero: 
# - presenza di menzioni ad altri utenti
# - presenza di collegamenti esterni
# - presenza di keyword relative a X (RT, FAV, quest'ultima non presente nei nostri dati)
# - presenza di stop-words in lingua italiana dalla lista fornita dal NaturalLanguageToolKit

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        if t.startswith('@') and len(t) > 1 or t.startswith('http') or t in X_keywords or t.lower() in stopwords_it:
            continue
        else:
            new_text.append(t)
    return " ".join(new_text)

def getHashtags(text):
    hashtags = []
    for t in text.split(" "):
        if t.startswith('#') and len(t) > 1:
            hashtags.append(t)
        else:
            continue
    return hashtags

def removeHashTags(text):
    new_text = []
    for t in text.split(" "):
        if t.startswith('#') and len(t) > 1:
            continue
        else:
            new_text.append(t)
    return " ".join(new_text)

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3


### Task A: Preprocessing
Al fine di valutare soltanto le parti più importanti dei messaggi, con un contributo semantico più corposo, sono state prese diverse decisioni:
- Eliminazione delle menzioni degli utenti (nomi utenti già oscurati nel dataset utilizzato)
- Eliminazione delle stopword da NLTK
- Eliminazione delle keyword legate a X (RT per i retweet e FAV per i messaggi preferiti)
- Eliminazione link esterni
- Eliminazione hashtag

Controlliamo innanzitutto l'integrità dei dati di addestramento

In [ ]:
data.isna().sum()

Non è necessario fare imputazione in quanto non ci sono dati mancanti all'interno del dataset di addestramento.

Controlliamo anche velocemente quali sono le emozioni maggiormente espresse nei messaggi.

Possiamo vedere come i messaggi esprimano emozioni variegate (con un picco nei messaggi di fiducia/incoraggiamento e un minimo nei messaggi che esprimono timore o paura, fatto comprensibile dal momento che la maggior parte dei commenti riguarda trasmissioni in onda).

In [ ]:
plt.barh(emotions.columns, [emotions[column].sum() for column in emotions.columns])

Possiamo adesso procedere al preprocessing del testo.

In [ ]:
text_prep = text_prep.apply(preprocess)

Dopo questo primo passo, è sorto un dubbio riguardante l'utilità degli hashtag e la conseguente decisione sul mantenerli o meno. 
La scelta finale è stata quella di rimuoverli, dal momento che gli hashtag più diffusi almeno in questa prima fase non aggiungono molto valore al testo essendo relativi al topic (generalmente il nome del programma televisivo interessato).

Potrebbe essere interessante mantenere queste informazioni per fare uno studio di sentiment analysis differenziata per singolo programma televisivo, tuttavia non è il focus principale del progetto.

Sono stati rilevati in totale 1126 hashtag unici su 5990 totali (in riferimento ai dati di addestramento), con una distribuzione sbilanciata fortemente verso i 20-30 circa più usati, che sono risultati essere hashtag relativi appunto al nome della trasmissione televisiva oggetto del testo.

In [ ]:
hashtag_list = list(map(getHashtags, text_prep))
hashtags = []

for h in hashtag_list:
    hashtags.extend(h)

hashtags = list(sorted(hashtags))
unique_hashtags = list(set(hashtags))
hasht_count = [hashtags.count(h) for h in unique_hashtags]

sorted_h = [x for _, x in sorted(zip(hasht_count, unique_hashtags), reverse=True)]
sorted_h_c = list(sorted(hasht_count,reverse=True))

In [ ]:
plt.barh(sorted_h[:25], sorted_h_c[:25])

In [ ]:
text_prep = text_prep.apply(removeHashTags)

Alcuni messaggi sono composti soltanto da menzioni 
o semplici hashtag, per cui facciamo una veloce pulizia prima di procedere 
(al contrario, alcuni messaggi non verrebbero processati dal tokenizer, causando degli errori)

In [ ]:
for i in range(len(text_prep)):
    if(len(text_prep[i]) == 0):
        text_prep.drop(i, axis=0, inplace=True)
        emotions_prep.drop(i, axis=0, inplace=True)

Prima di procedere alla classificazione vera e propria, dobbiamo adattare il tokenizer alla presenza delle emoji nel testo, elemento importante per l'interpretazione dei messaggi.

Facciamo quindi una ricerca di emoji ed emoticons presenti e aggiungiamole al tokenizer tramite la funzione add_tokens(), per poi fare un resize del modello e completare questo step.

In [ ]:
bulk_emoji = emot.bulk_emoji(text_prep)

unique_emoji = []

for i in range(len(bulk_emoji)):
    res = bulk_emoji[i]
    if res['flag']:
        unique_emoji.extend(res['value'])

unique_emoji = list(set(unique_emoji))

emoji_to_add = set(unique_emoji) - set(tokenizer.vocab.keys())

In [ ]:
bulk_emoticons = emot.bulk_emoticons(text_prep)

unique_emoticons = []

for i in range(len(bulk_emoticons)):
    res = bulk_emoticons[i]
    if res['flag']:
        unique_emoticons.extend(res['value'])

unique_emoticons = list(set(unique_emoticons))

emoticons_to_add = set(unique_emoticons) - set(tokenizer.vocab.keys())

In [ ]:
new_tokens = []
new_tokens.extend(emoji_to_add)
new_tokens.extend(emoticons_to_add)

In [ ]:
print(f"Adding: {len(new_tokens)} emoji/emoticons")

In [ ]:
tokenizer.add_tokens(list(new_tokens))
model.resize_token_embeddings(len(tokenizer))

Vediamo adesso di stabilire brevemente una lunghezza massima in parole dei messaggi.

Possiamo vedere come la stragrande maggioranza dei messaggi abbia una lunghezza inferiore a 25-30 parole, per cui useremo 30 come tetto massimo.

In [ ]:
plt.hist([len(t.split(' ')) for t in text_prep])
MAX_LEN = 30

Vediamo come appare il testo dei messaggi dopo questa operazione di pulizia

In [ ]:
print(text[5], '\n',text_prep[5])

## Task A: Classificazione

Arrivati a questo punto, dobbiamo per prima cosa addestrare il nostro classificatore per il task che dovrà svolgere.

La struttura adottata sarà composta da un BTE (Bidirectional Transformer Encoder) per l'encoding degli embeddings generati dal tokenizer.
A questa base aggiungiamo un layer dense e infine una sigmoide che ci permette di fare la classificazione multi-label richiesta.

Procediamo a passare tutti i dati etichettati al nostro classificatore 

In [ ]:
device = ("cpu")

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = ("cuda")

print(f"Using {device} device")

In [ ]:
class _Dataset(Dataset):
    def __init__(self, inputs, labels, tokenizer, MAX_LEN, transform=None):
        self.transform = transform
        self.inputs = inputs
        self.labels = [torch.tensor(l) for l in labels]           
        self.tokenizer = tokenizer
        self.MAX_LEN = MAX_LEN
      
    def __len__(self):
        return len(self.inputs)       

    def __getitem__(self, idx):
        inputs = tokenizer(self.inputs.iloc[idx], add_special_tokens=True,
                            return_tensors='pt', padding='max_length',
                            max_length = self.MAX_LEN, truncation=True) 
        labels = self.labels[idx]
        return {
            'ids': inputs['input_ids'],
            'mask': inputs['attention_mask'],
            'token_type_ids': inputs['token_type_ids'],
            'labels': labels
        } 

In [ ]:
class BTEClassifier(nn.Module):
    def __init__(self, dropout, num_classes, bte):
        super(BTEClassifier, self).__init__()
        self.bte = bte # Blocco BTE - Bidirectional Transformer Encoder
        self.classifier = nn.Sequential(
            nn.Dropout(dropout), # Layer di dropout,
            nn.Linear(768, num_classes), # Layer dense su cui fare fine tuning
            nn.Sigmoid() 
        )

    def forward(self, ids, mask, token_type_ids):
        output = self.bte(ids, attention_mask = mask, token_type_ids = token_type_ids).last_hidden_state
        output = output[:,0,:]
        output = self.classifier(output)
        return output

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
def train(model, dataloader, loss, accuracy, optimizer, device):
    model.train()

    epoch_acc = 0
    epoch_loss = 0

    for d in tqdm(dataloader):
        (ids, mask, token_type_ids, labels_) = d.values()

        optimizer.zero_grad()

        input_ids = ids.squeeze(1).to(device)
        input_mask = mask.squeeze(1).to(device)
        input_token_type_ids = token_type_ids.squeeze(1).to(device)
        labels_ = labels_.squeeze(1).to(device).float()

        output = model(input_ids, input_mask, input_token_type_ids)

        batch_loss = loss(output, labels_)
        batch_loss.backward()
        optimizer.step()

        epoch_loss += batch_loss.item()
        epoch_acc += accuracy(output, labels_).item()
        
        input_ids = input_ids.detach().cpu()
        input_mask = input_mask.detach().cpu()
        input_token_type_ids = input_token_type_ids.detach().cpu()
        labels_ = labels_.detach().cpu()
        
    return epoch_loss/len(dataloader), epoch_acc/len(dataloader), model

In [ ]:
def val(model, dataloader, loss, accuracy, device, best_acc, best_weights):
    model.eval()

    epoch_acc = 0
    epoch_loss = 0
    
    with torch.no_grad():
        for d in tqdm(dataloader):
            (ids, mask, token_type_ids, labels_) = d.values()

            input_ids = ids.squeeze(1).to(device)
            input_mask = mask.squeeze(1).to(device)
            input_token_type_ids = token_type_ids.squeeze(1).to(device)
            labels_ = labels_.squeeze(1).to(device).float()

            output = model(input_ids, input_mask, input_token_type_ids)
            
            batch_loss = loss(output, labels_)
            epoch_loss += batch_loss.item()

            epoch_acc += accuracy(output, labels_).item()
            
            input_ids = input_ids.detach().cpu()
            input_mask = input_mask.detach().cpu()
            input_token_type_ids = input_token_type_ids.detach().cpu()
            labels_ = labels_.detach().cpu()

        acc = float(epoch_acc/len(dataloader))
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
    
    return epoch_loss/len(dataloader), epoch_acc/len(dataloader), best_acc, best_weights, model

In [ ]:
def evaluate(model, epochs, optimizer, device, train_data, val_data,
               batch_size, accuracy, loss, early_stopper):

  train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
  val_dataloader = DataLoader(val_data, batch_size=batch_size)

  # liste dei valori di loss e accuracy epoca per epoca per il plot
  train_loss = []
  val_loss = []

  train_acc = []
  val_acc = []
  best_acc = - np.inf
  best_weights = None

  # Ciclo di addestramento 
  for epoch in tqdm(range(1, epochs+1)):
    epoch_train_loss, epoch_train_acc, model = train(model, train_dataloader, loss, accuracy, optimizer, device)
    train_loss.append(epoch_train_loss)
    train_acc.append(epoch_train_acc)
    
    epoch_val_loss, epoch_val_acc, best_acc, best_weights, model = val(model, val_dataloader, loss, accuracy, device, best_acc, best_weights)
    val_loss.append(epoch_val_loss)
    val_acc.append(epoch_val_acc)

    if early_stopper.early_stop(epoch_val_loss):             
      break

    print(f"\nTrain loss: {epoch_train_loss:6.4f} Val loss: {epoch_val_loss:6.4f}")
    print(f"Train accuracy: {(epoch_train_acc):6.4f} Val accuracy: {(epoch_val_acc):6.4f}")

  return train_loss, val_loss, train_acc, val_acc, best_acc, best_weights

In [ ]:
def predict(model, dataset):
    dataloader = DataLoader(dataset)
    predictions = []

    with torch.no_grad():
        for d in tqdm(dataloader):
            (ids, mask, token_type_ids, labels_) = d.values()

            input_ids = ids.squeeze(1).to(device)
            input_mask = mask.squeeze(1).to(device)
            input_token_type_ids = token_type_ids.squeeze(1).to(device)
            labels_ = labels_.squeeze(1).to(device).float()

            output = model(input_ids, input_mask, input_token_type_ids)

            output = output.squeeze(0).detach().cpu().tolist()
            pred = [1 if o > .5 else 0 for o in output]
            input_ids= input_ids.detach().cpu()
            input_mask = input_mask.detach().cpu()
            input_token_type_ids = input_token_type_ids.detach().cpu()
            labels_ = labels_.detach().cpu()
            predictions.append(pred)
        
    return predictions

In [ ]:
hyperparameters = {
    'dropout' : [0.1, 0.2, 0.3, 0.4],
    'batch_size' : [4, 8, 16, 32],
    'epochs' : [5, 10, 20],
    'lr' : [1e-06, 1e-05, 1e-04, 1e-03],
    'max_len' : MAX_LEN,
    'num_classes' : 10,
}

loss = nn.BCELoss()

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(text, emotions.to_numpy(), test_size=0.3)
train_dataset = _Dataset(X_train, Y_train, tokenizer, hyperparameters['max_len'])
val_dataset = _Dataset(X_val, Y_val, tokenizer, hyperparameters['max_len'])

X_train_prep, X_val_prep, Y_train_prep, Y_val_prep = train_test_split(text_prep, emotions_prep.to_numpy(), test_size=0.3)
train_dataset_prep = _Dataset(X_train_prep, Y_train_prep, tokenizer, hyperparameters['max_len'])
val_dataset_prep = _Dataset(X_val_prep, Y_val_prep, tokenizer, hyperparameters['max_len'])

In [ ]:
hist = []
runs = []
n_runs = 5
run = None
i = 0

if not os.path.exists('hist.pkl'):
    while i < n_runs:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        accuracy = MultilabelAccuracy(num_labels=10).to(device)

        dropout = rand.choice(hyperparameters['dropout'])
        lr = rand.choice(hyperparameters['lr'])
        epochs = rand.choice(hyperparameters['epochs'])
        batch_size = rand.choice(hyperparameters['batch_size'])
        run = [dropout, lr, epochs, batch_size]

        if run in runs:
            continue

        i = i + 1

        runs.append(run)
        
        early_stopper = EarlyStopper(patience=1, min_delta=0.008)
        classifier = BTEClassifier(dropout, hyperparameters['num_classes'], model).to(device)
        optimizer = Adam(classifier.parameters(), lr=lr)

        train_loss, val_loss, train_acc, val_acc, best_acc, best_weights = evaluate(
            classifier, 
            epochs, 
            optimizer, 
            device, 
            train_dataset, 
            val_dataset,
            batch_size, 
            accuracy, 
            loss,
            early_stopper
        )

        classifier = classifier.cpu()
        accuracy = accuracy.cpu()
        
        hist.append(
            {
                "epochs" : epochs,
                "dropout" : dropout,
                "lr" : lr,
                "batch_size" : batch_size,
                "train_loss" : train_loss,
                "val_loss" : val_loss,
                "train_acc" : train_acc,
                "val_acc" : val_acc,
                "best_acc" : best_acc,
                "weights" : best_weights
            }
        )
    with open('hist.pkl', 'wb') as handle:
        save = {
            "hist" : hist,
            "runs" : runs
        }
        pickle.dump(save, handle)
else:
    f = open("hist.pkl",'rb')
    save = pickle.load(f)
    hist = save['hist']
    runs = save['runs']

In [ ]:
fig, (ax) = plt.subplots( nrows=2, ncols=3, figsize=(10, 10))
n_epochs = list(range(0, 22, 2))

for i in range(2):
    for j in range(2):
        ax[i][j].set_title(f"Lr:{lr}, epochs:{hist[i + j]['epochs']}, \ndropout:{hist[i + j]['dropout']}, batch:{hist[i + j]['batch_size']}")
        ax[i][j].plot(hist[i + j]['train_loss'], label='training loss')
        ax[i][j].plot(hist[i + j]['val_loss'], label='val loss')
        ax[i][j].legend(loc='lower right')
        ax[i][j].set_xticks(n_epochs)

i = i - 1
j = j + 1
ax[i][j].set_title(f"Lr:{lr}, epochs:{hist[i + j]['epochs']}, \ndropout:{hist[i + j]['dropout']}, batch:{hist[i + j]['batch_size']}")
ax[i][j].plot(hist[i + j]['train_loss'], label='training loss')
ax[i][j].plot(hist[i + j]['val_loss'], label='val loss')
ax[i][j].legend(loc='lower right')
ax[i][j].set_xticks(n_epochs)

plt.show()
fig.savefig('train_A.png')
plt.close(fig)

In [ ]:
prep_hist = []
prep_runs = []
run = None
i = 0

if not os.path.exists('prep_hist.pkl'):
    while i < n_runs:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        accuracy = MultilabelAccuracy(num_labels=10).to(device)
        
        dropout = rand.choice(hyperparameters['dropout'])
        lr = rand.choice(hyperparameters['lr'])
        epochs = rand.choice(hyperparameters['epochs'])
        batch_size = rand.choice(hyperparameters['batch_size'])

        run = [dropout, lr, epochs, batch_size]

        if run in prep_runs:
            continue

        i = i + 1

        prep_runs.append(run)
        
        early_stopper = EarlyStopper(patience=1, min_delta=0.008)
        classifier = BTEClassifier(dropout, hyperparameters['num_classes'], model).to(device)
        optimizer = Adam(classifier.parameters(), lr=lr)

        train_loss, val_loss, train_acc, val_acc, best_acc, best_weights = evaluate(
            classifier, 
            epochs, 
            optimizer, 
            device, 
            train_dataset_prep, 
            val_dataset_prep,
            batch_size, 
            accuracy, 
            loss,
            early_stopper
        )

        classifier = classifier.cpu()
        accuracy = accuracy.cpu()

        prep_hist.append(
            {
                "epochs" : epochs,
                "dropout" : dropout,
                "lr" : lr,
                "batch_size" : batch_size,
                "train_loss" : train_loss,
                "val_loss" : val_loss,
                "train_acc" : train_acc,
                "val_acc" : val_acc,
                "best_acc" : best_acc,
                "weights" : best_weights
            }
        )

    with open('prep_hist.pkl', 'wb') as handle:
        prep_save = {
            "hist" : prep_hist,
            "runs" : runs
        }
        pickle.dump(prep_save, handle)
else:
    f = open("prep_hist.pkl",'r')
    prep_save = pickle.load(f)
    prep_hist = prep_save['hist']
    prep_runs = prep_save['runs']

In [ ]:
fig, ax = plt.subplots( nrows=2, ncols=3, figsize=(10, 10))
n_epochs = list(range(0, 22, 2))

for i in range(2):
    for j in range(2):
        ax[i][j].set_title(f"Lr:{lr}, epochs:{prep_hist[i + j]['epochs']}, \ndropout:{prep_hist[i + j]['dropout']}, batch:{prep_hist[i + j]['batch_size']}")
        ax[i][j].plot(prep_hist[i + j]['train_loss'], label='training loss')
        ax[i][j].plot(prep_hist[i + j]['val_loss'], label='val loss')
        ax[i][j].legend(loc='lower right')
        ax[i][j].set_xticks(n_epochs)
i = i - 1
j = j + 1
ax[i][j].set_title(f"Lr:{lr}, epochs:{prep_hist[i + j]['epochs']}, \ndropout:{prep_hist[i + j]['dropout']}, batch:{prep_hist[i + j]['batch_size']}")
ax[i][j].plot(prep_hist[i + j]['train_loss'], label='training loss')
ax[i][j].plot(prep_hist[i + j]['val_loss'], label='val loss')
ax[i][j].legend(loc='lower right')
ax[i][j].set_xticks(n_epochs)

plt.show()
fig.savefig('train_prep_A.png')
plt.close(fig)

In [ ]:
test = pd.read_csv("release\emit_2023-test-labels\emit_test.csv")
test_prep = test.copy()

test_ood = pd.read_csv("release\emit_2023-test-labels\emit_test_ood.csv")
test_ood_prep = test_ood.copy()

test.drop(["Topic", "Direction"], axis=1, inplace=True)
test_ood.drop(["Topic", "Direction"], axis=1, inplace=True)

test_prep.drop(["Topic", "Direction"], axis=1, inplace=True)
test_ood_prep.drop(["Topic", "Direction"], axis=1, inplace=True)

test_prep['text'] = test_prep['text'].apply(preprocess).apply(removeHashTags)
test_ood_prep['text'] = test_ood_prep['text'].apply(preprocess).apply(removeHashTags)

test_prep = test_prep.loc[test_prep['text'].str.len() > 0]
test_ood_prep = test_ood_prep.loc[test_ood_prep['text'].str.len() > 0]

test_prep_dataset = _Dataset(test_prep['text'], test_prep.drop(['id','text'], axis=1).to_numpy(), tokenizer, hyperparameters['max_len'])
test_ood_prep_dataset = _Dataset(test_ood_prep['text'], test_ood_prep.drop(['id','text'], axis=1).to_numpy(), tokenizer, hyperparameters['max_len'])

test_dataset = _Dataset(test['text'], test.drop(['id','text'], axis=1).to_numpy(), tokenizer, hyperparameters['max_len'])
test_ood_dataset = _Dataset(test_ood['text'], test_ood.drop(['id','text'], axis=1).to_numpy(), tokenizer, hyperparameters['max_len'])

In [ ]:
classifier = classifier.to(device)
classes = emotions.columns

In [ ]:
if not os.path.exists('heatmap_A.png'):
    preds = []
    preds_ood = []
    for h in hist:
        weights = h['weights']

        classifier.load_state_dict(weights)

        preds.append(predict(classifier, test_dataset))
        preds_ood.append(predict(classifier, test_ood_dataset))

    target = test[classes].to_numpy()
    target_ood = test_ood[classes].to_numpy()

    f1_scores = [f1(pred, target, average='weighted') for pred in preds]
    f1_ood_scores = [f1(ood_pred, target_ood, average='weighted') for ood_pred in preds_ood]

    best_index = f1_scores.index(max(f1_scores))
    best_hyperp = hist[best_index]
    best_index_ood = f1_ood_scores.index(max(f1_ood_scores))
    best_ood_hyperp = hist[best_index_ood]

    if not os.path.exists('best_weights_a.pt'):
        torch.save(best_hyperp['weights'], 'best_weights_a.pt')
        torch.save(best_ood_hyperp['weights'], 'best_weights_ood_a.pt')

    fig, ax = plt.subplots(nrows=2, figsize=(10, 10))

    ax[0].set_title('In domain')
    ax[1].set_title('OOD')
    report = cr(preds[best_index], target, target_names=classes, output_dict=True)
    report_ood = cr(preds_ood[best_index], target, target_names=classes, output_dict=True)
    sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True, ax=ax[0]).get_figure()
    sns.heatmap(pd.DataFrame(report_ood).iloc[:-1, :].T, annot=True, ax=ax[1]).get_figure()
    fig.savefig('heatmap_A.png')

    recap = open("recap.txt", "w")
    txt = [
        "lr;epochs;batch_size;dropout;ood;f1-score", 
        f"{best_hyperp['lr']};{best_hyperp['epochs']};{best_hyperp['batch_size']};{best_hyperp['dropout']};False;{max(f1_scores)}",
        f"{best_ood_hyperp['lr']};{best_ood_hyperp['epochs']};{best_ood_hyperp['batch_size']}; {best_ood_hyperp['dropout']};True;{max(f1_ood_scores)}"
    ]
    recap.write("\n".join(txt))
    recap.close()

In [ ]:
if not os.path.exists('heatmap_prep_A.png'):
    preds_prep = []
    preds_ood_prep = []
    for h in prep_hist:
        weights = h['weights']

        classifier.load_state_dict(weights)

        preds_prep.append(predict(classifier, test_prep_dataset))
        preds_ood_prep.append(predict(classifier, test_ood_prep_dataset))

    target_prep = test_prep[classes].to_numpy()
    target_prep_ood = test_ood_prep[classes].to_numpy()

    f1_prep_scores = [f1(pred_prep, target_prep, average='weighted') for pred_prep in preds_prep]
    f1_prep_ood_scores = [f1(ood_prep_pred, target_prep_ood, average='weighted') for ood_prep_pred in preds_ood_prep]

    best_prep_index = f1_prep_scores.index(max(f1_prep_scores))
    best_prep_hyperp = prep_hist[best_prep_index]

    best_prep_index_ood = f1_prep_ood_scores.index(max(f1_prep_ood_scores))
    best_prep_ood_hyperp = prep_hist[best_prep_index_ood]

    if not os.path.exists('best_weights_prep_a.pt'):
        torch.save(best_prep_hyperp['weights'], 'best_weights_prep_a.pt')
        torch.save(best_prep_ood_hyperp['weights'], 'best_weights_ood_prep_a.pt')

    fig, ax = plt.subplots(nrows=2, figsize=(10, 10))

    ax[0].set_title('In domain')
    ax[1].set_title('OOD')
    report_prep = cr(preds_prep[best_prep_index], target_prep, target_names=classes, output_dict=True)
    report_ood_prep = cr(preds_ood_prep[best_prep_index], target_prep, target_names=classes, output_dict=True)
    sns.heatmap(pd.DataFrame(report_prep).iloc[:-1, :].T, annot=True, ax=ax[0]).get_figure()
    sns.heatmap(pd.DataFrame(report_ood_prep).iloc[:-1, :].T, annot=True, ax=ax[1]).get_figure()
    fig.savefig('heatmap_prep_A.png')
    
    recap = open("recap_prep.txt", "w")
    txt = [
        "lr;epochs;batch_size;dropout;ood;f1-score", 
        f"{best_prep_hyperp['lr']};{best_prep_hyperp['epochs']};{best_prep_hyperp['batch_size']};{best_prep_hyperp['dropout']};False;{max(f1_prep_scores)}",
        f"{best_prep_ood_hyperp['lr']};{best_prep_ood_hyperp['epochs']};{best_prep_ood_hyperp['batch_size']}; {best_prep_ood_hyperp['dropout']};True;{max(f1_prep_ood_scores)}"
    ]
    recap.write("\n".join(txt))
    recap.close()